# Step 3A: Model Scoring evaluation

Using the labeled feature data set constructed in the `./notebooks/2b_feature_engineering` Jupyter notebook, this notebook loads the data scores the observations. 

**Note:** This notebook will take about 2-4 minutes to execute all cells, depending on the compute configuration you have setup.

In [2]:
# import the libraries
import time

# For some data handling
import numpy as np
from pyspark.ml import PipelineModel
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

# Time the notebook execution. 
# This will only make sense if you "Run all cells"
tic = time.time()

# We will store and read each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# The scoring uses the same feature engineering script used to train the model
scoring_table = 'scoring_input'
results_table = 'results_output'
start_date = "2015-12-30"
end_date = "2016-04-30"

model = 'RandomForest' # Use 'DecisionTree' or 'RandomForest'

In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text("scoring_data", scoring_table)
dbutils.widgets.text("results_data", results_table)

dbutils.widgets.text("start_date", start_date)

dbutils.widgets.text("to_date", end_date)

dbutils.widgets.text("model", model)

We need to run the feature engineering on the data we're interested in scoring. This should be the same feature engineering steps we used to train the model. If you do this in the data base, this next cell could be a simple select statement. For this example, we used a parameterized notebook to do feature engineering, so we can just rerun the notebook with parameters to operate on the raw data tables.

In [5]:
# This cell must complete in less than 10minutes, or databricks will kill it. So we set the time out to 9.5min = 9.5*60 sec = 570 seconds
dbutils.notebook.run("2a_feature_engineering", 600, 
                     {"features_table": dbutils.widgets.get("scoring_data"), 
                      "start_date": dbutils.widgets.get("start_date"),  
                      "to_date": dbutils.widgets.get("to_date")})

Load the data and dump a short summary of the resulting DataFrame.

In [7]:
sqlContext.refreshTable(dbutils.widgets.get("scoring_data")) 

score_data = spark.sql("SELECT * FROM " + dbutils.widgets.get("scoring_data"))

print(score_data.count())

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaing feature names from the data
input_features = score_data.columns

# We'll use the known label, key variables and 
# a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Remove the extra names if that are in the input_features list
input_features = [x for x in input_features if x not in set(remove_names)]

input_features
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')

# assemble features
score_data = va.transform(score_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(score_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(score_data)

print(score_data.count())
score_data.toPandas().head(20)

6005
6005
 Out[ 4 ]: 
 machineID dt_truncated label_e \
0 27 2016-01-01 12:00:00 0.0 
1 27 2016-01-01 00:00:00 0.0 
2 27 2015-12-31 12:00:00 0.0 
3 27 2015-12-31 00:00:00 0.0 
4 27 2015-12-30 12:00:00 0.0 
5 27 2015-12-30 00:00:00 0.0 
6 63 2016-01-01 12:00:00 0.0 
7 63 2016-01-01 00:00:00 0.0 
8 63 2015-12-31 12:00:00 0.0 
9 63 2015-12-31 00:00:00 0.0 
10 63 2015-12-30 12:00:00 0.0 
11 63 2015-12-30 00:00:00 0.0 
12 158 2016-01-01 12:00:00 0.0 
13 158 2016-01-01 00:00:00 0.0 
14 158 2015-12-31 12:00:00 0.0 
15 158 2015-12-31 00:00:00 0.0 
16 158 2015-12-30 12:00:00 0.0 
17 158 2015-12-30 00:00:00 0.0 
18 389 2016-01-01 12:00:00 0.0 
19 389 2016-01-01 00:00:00 0.0 

 features 
0 (162.374561325, 445.713043834, 103.468531996, ... 
1 [169.63423645, 448.824823839, 100.134285273, 4... 
2 [168.028923157, 462.552245357, 97.5496715183, ... 
3 [162.926819659, 446.38100527, 99.8598322727, 3... 
4 [161.979750244, 445.396520667, 102.676947074, ... 
5 [167.273061702, 414.154289045, 97.0787477664, ... 
6 (170.665340293, 429.879779516, 102.52265497, 3... 
7 [170.909065392, 431.447522699, 101.58237207, 3... 
8 [168.68383159, 444.657960848, 100.900896359, 4... 
9 [172.052881411, 463.527143784, 96.3051763888, ... 
10 [169.785873523, 431.41255335, 99.9372850746, 3... 
11 [169.744079356, 465.405198219, 108.317072627, ... 
12 (171.493828712, 382.369420732, 99.0673772021, ... 
13 [168.186955023, 373.164849155, 100.27292124, 3... 
14 [171.302368793, 384.468098503, 101.990866038, ... 
15 [172.950717619, 439.978358371, 101.819695031, ... 
16 [164.436920443, 421.288136157, 103.451645545, ... 
17 [147.077021191, 452.425483615, 103.650360209, ... 
18 (174.812980875, 373.152557113, 100.378537101, ... 
19 [174.281654656, 370.171992292, 102.049819283, ...

In [8]:
type(score_data)

Out[ 5 ]: pyspark.sql.dataframe.DataFrame

# Prepare the Training/Testing data

A fundamental practice in machine learning is to calibrate and test your model parameters on data that has not been used to train the model. Evaluation of the model requires splitting the available data into a training portion, a calibration portion and an evaluation portion. Typically, 80% of data is used to train the model and 10% each to calibrate any parameter selection and evaluate your model.

In general random splitting can be used, but since time series data have an inherent correlation between observations. For predictive maintenance problems, a time-dependent spliting strategy is often a better approach to estimate performance. For a time-dependent split, a single point in time is chosen, the model is trained on examples up to that point in time, and validated on the examples after that point. This simulates training on current data and score data collected in the future data after the splitting point is not known. However, care must be taken on labels near the split point. In this case, feature records within 7 days of the split point can not be labeled as a failure, since that is unobserved data. 

In the following code blocks, we split the data at a single point to train and evaluate this model.

Spark models require a vectorized data frame. We transform the dataset here and then split the data into a training and test set. We use this split data to train the model on 9 months of data (training data), and evaluate on the remaining 3 months (test data) going forward.

# Classification models

A particualar troubling behavior in predictive maintenance is machine failures are usually rare occurrences compared to normal operation. This is fortunate for the business as maintenance and saftey issues are few, but causes an imbalance in the label distribution. This imbalance leads to poor performance as algorithms tend to classify majority class examples at the expense of minority class, since the total misclassification error is much improved when majority class is labeled correctly. This causes low recall or precision rates, although accuracy can be high. It becomes a larger problem when the cost of false alarms is very high. To help with this problem, sampling techniques such as oversampling of the minority examples can be used. These methods are not covered in this notebook. Because of this, it is also important to look at evaluation metrics other than accuracy alone.

We will build and compare two different classification model approaches:

 - **Decision Tree Classifier**: Decision trees and their ensembles are popular methods for the machine learning tasks of classification and regression. Decision trees are widely used since they are easy to interpret, handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions.

 - **Random Forest Classifier**: A random forest is an ensemble of decision trees. Random forests combine many decision trees in order to reduce the risk of overfitting. Tree ensemble algorithms such as random forests and boosting are among the top performers for classification and regression tasks.

We will to compare these models in the AML Workbench _runs_ screen. The next code block creates the model. You can choose between a _DecisionTree_ or _RandomForest_ by setting the 'model_type' variable. We have also included a series of model hyperparameters to guide your exploration of the model space.

To evaluate this model, we predict the component failures over the test data set. Since the test set has been created from data the model has not been seen before, it simulates future data. The evaluation then can be generalize to how the model could perform when operationalized and used to score the data in real time.

In [14]:
model_pipeline = PipelineModel.load("dbfs:/storage/models/" + dbutils.widgets.get("model") + ".pqt")

print("Model loaded")
model_pipeline

Model loaded
 Out[ 6 ]: PipelineModel_4b7bbfd4676e4dd3b642

In [15]:

# make predictions. The Pipeline does all the same operations on the test data
predictions = model_pipeline.transform(score_data)

# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas()
confuse.head()

Out[ 7 ]: 
 indexedLabel_prediction 0.0 1.0 2.0 3.0 4.0
0 0.0 5785 28 53 24 2
1 1.0 9 11 0 0 0
2 2.0 10 0 12 0 0
3 3.0 8 1 0 31 0
4 4.0 18 1 0 0 12

The confusion matrix lists each true component failure in rows and the predicted value in columns. Labels numbered 0.0 corresponds to no component failures. Labels numbered 1.0 through 4.0 correspond to failures in one of the four components in the machine. As an example, the third number in the top row indicates how many days we predicted component 2 would fail, when no components actually did fail. The second number in the second row, indicates how many days we correctly predicted a component 1 failure within the next 7 days.

We read the confusion matrix numbers along the diagonal as correctly classifying the component failures. Numbers above the diagonal indicate the model incorrectly predicting a failure when non occured, and those below indicate incorrectly predicting a non-failure for the row indicated component failure.

When evaluating classification models, it is convenient to reduce the results in the confusion matrix into a single performance statistic. However, depending on the problem space, it is impossible to always use the same statistic in this evaluation. Below, we calculate four such statistics.

- **Accuracy**: reports how often we correctly predicted the labeled data. Unfortunatly, when there is a class imbalance (a large number of one of the labels relative to others), this measure is biased towards the largest class. In this case non-failure days.

Because of the class imbalance inherint in predictive maintenance problems, it is better to look at the remaining statistics instead. Here positive predictions indicate a failure.

- **Precision**: Precision is a measure of how well the model classifies the truely positive samples. Precision depends on falsely classifying negative days as positive.

- **Recall**: Recall is a measure of how well the model can find the positive samples. Recall depends on falsely classifying positive days as negative.

- **F1**: F1 considers both the precision and the recall. F1 score is the harmonic average of precision and recall. An F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0.

These metrics make the most sense for binary classifiers, though they are still useful for comparision in our multiclass setting. Below we calculate these evaluation statistics for the selected classifier, and post them back to the AML workbench run time page for tracking between experiments.

In [17]:
# select (prediction, true label) and compute test error
# select (prediction, true label) and compute test error
# True positives - diagonal failure terms 
tp = confuse['1.0'][1]+confuse['2.0'][2]+confuse['3.0'][3]+confuse['4.0'][4]

# False positves - All failure terms - True positives
fp = np.sum(np.sum(confuse[['1.0', '2.0','3.0','4.0']])) - tp

# True negatives 
tn = confuse['0.0'][0]

# False negatives total of non-failure column - TN
fn = np.sum(np.sum(confuse[['0.0']])) - tn

# Accuracy is diagonal/total 
acc_n = tn + tp
acc_d = np.sum(np.sum(confuse[['0.0','1.0', '2.0','3.0','4.0']]))
acc = acc_n/acc_d

# Calculate precision and recall.
prec = tp/(tp+fp)
rec = tp/(tp+fn)

# Print the evaluation metrics to the notebook
print("Accuracy = %g" % acc)
print("Precision = %g" % prec)
print("Recall = %g" % rec )
print("F1 = %g" % (2.0 * prec * rec/(prec + rec)))
print("")

Accuracy = 0.974355
Precision = 0.377143
Recall = 0.594595
F1 = 0.461538

Remember that this is a simulated data set. We would expect a model built on real world data to behave very differently. The accuracy may still be close to one, but the precision and recall numbers would be much lower.

In [19]:
predictions.toPandas().head(20)

Out[ 9 ]: 
 machineID dt_truncated label_e \
0 27 2016-01-01 12:00:00 0.0 
1 27 2016-01-01 00:00:00 0.0 
2 27 2015-12-31 12:00:00 0.0 
3 27 2015-12-31 00:00:00 0.0 
4 27 2015-12-30 12:00:00 0.0 
5 27 2015-12-30 00:00:00 0.0 
6 63 2016-01-01 12:00:00 0.0 
7 63 2016-01-01 00:00:00 0.0 
8 63 2015-12-31 12:00:00 0.0 
9 63 2015-12-31 00:00:00 0.0 
10 63 2015-12-30 12:00:00 0.0 
11 63 2015-12-30 00:00:00 0.0 
12 158 2016-01-01 12:00:00 0.0 
13 158 2016-01-01 00:00:00 0.0 
14 158 2015-12-31 12:00:00 0.0 
15 158 2015-12-31 00:00:00 0.0 
16 158 2015-12-30 12:00:00 0.0 
17 158 2015-12-30 00:00:00 0.0 
18 389 2016-01-01 12:00:00 0.0 
19 389 2016-01-01 00:00:00 0.0 

 features indexedLabel \
0 (162.374561325, 445.713043834, 103.468531996, ... 0.0 
1 [169.63423645, 448.824823839, 100.134285273, 4... 0.0 
2 [168.028923157, 462.552245357, 97.5496715183, ... 0.0 
3 [162.926819659, 446.38100527, 99.8598322727, 3... 0.0 
4 [161.979750244, 445.396520667, 102.676947074, ... 0.0 
5 [167.273061702, 414.154289045, 97.0787477664, ... 0.0 
6 (170.665340293, 429.879779516, 102.52265497, 3... 0.0 
7 [170.909065392, 431.447522699, 101.58237207, 3... 0.0 
8 [168.68383159, 444.657960848, 100.900896359, 4... 0.0 
9 [172.052881411, 463.527143784, 96.3051763888, ... 0.0 
10 [169.785873523, 431.41255335, 99.9372850746, 3... 0.0 
11 [169.744079356, 465.405198219, 108.317072627, ... 0.0 
12 (171.493828712, 382.369420732, 99.0673772021, ... 0.0 
13 [168.186955023, 373.164849155, 100.27292124, 3... 0.0 
14 [171.302368793, 384.468098503, 101.990866038, ... 0.0 
15 [172.950717619, 439.978358371, 101.819695031, ... 0.0 
16 [164.436920443, 421.288136157, 103.451645545, ... 0.0 
17 [147.077021191, 452.425483615, 103.650360209, ... 0.0 
18 (174.812980875, 373.152557113, 100.378537101, ... 0.0 
19 [174.281654656, 370.171992292, 102.049819283, ... 0.0 

 indexedFeatures \
0 (162.374561325, 445.713043834, 103.468531996, ... 
1 [169.63423645, 448.824823839, 100.134285273, 4... 
2 [168.028923157, 462.552245357, 97.5496715183, ... 
3 [162.926819659, 446.38100527, 99.8598322727, 3... 
4 [161.979750244, 445.396520667, 102.676947074, ... 
5 [167.273061702, 414.154289045, 97.0787477664, ... 
6 (170.665340293, 429.879779516, 102.52265497, 3... 
7 [170.909065392, 431.447522699, 101.58237207, 3... 
8 [168.68383159, 444.657960848, 100.900896359, 4... 
9 [172.052881411, 463.527143784, 96.3051763888, ... 
10 [169.785873523, 431.41255335, 99.9372850746, 3... 
11 [169.744079356, 465.405198219, 108.317072627, ... 
12 (171.493828712, 382.369420732, 99.0673772021, ... 
13 [168.186955023, 373.164849155, 100.27292124, 3... 
14 [171.302368793, 384.468098503, 101.990866038, ... 
15 [172.950717619, 439.978358371, 101.819695031, ... 
16 [164.436920443, 421.288136157, 103.451645545, ... 
17 [147.077021191, 452.425483615, 103.650360209, ... 
18 (174.812980875, 373.152557113, 100.378537101, ... 
19 [174.281654656, 370.171992292, 102.049819283, ... 

 rawPrediction \
0 [194.818080952, 2.38343852109, 1.61969395882, ... 
1 [195.497549708, 1.9667072481, 1.4855159663, 0.... 
2 [195.302874731, 2.15297367665, 1.45020654836, ... 
3 [195.395743401, 2.11349766949, 1.46228862084, ... 
4 [194.984577984, 2.41947518418, 1.54131393302, ... 
5 [164.346557514, 29.7588608332, 3.5714182125, 1... 
6 [190.827962879, 3.09262503493, 2.58638885194, ... 
7 [192.828662759, 2.17306795386, 1.50501273278, ... 
8 [192.903016667, 2.02545719314, 1.50304922475, ... 
9 [192.728249649, 2.03981076619, 1.55680835096, ... 
10 [192.965996174, 2.07607150943, 1.45782071013, ... 
11 [174.651251129, 0.698327609769, 1.73962681977,... 
12 [155.108324675, 38.7443723342, 2.48626059478, ... 
13 [124.652803419, 73.0339095098, 1.10925205227, ... 
14 [143.650334865, 53.3580536805, 0.785911344795,... 
15 [193.74792578, 2.63516315764, 1.50327102927, 1... 
16 [162.74146305, 32.3064496173, 2.51446443785, 1... 
17 [190.422246841, 2.06424539704, 1.16495149178, ... 
18 [153.318897025, 40.6819908401, 3.80577071264, ... 
19 [110.903867526, 83.5105953164, 1.9

In [20]:
print(predictions.summary())

DataFrame[summary: string, machineID: string, label_e: string, indexedLabel: string, prediction: string]

In [21]:
predictions.explain()

== Physical Plan ==
*(1) Project [machineID#42203L, dt_truncated#42204, label_e#42242, features#42329, indexedLabel#42432, indexedFeatures#42439, rawPrediction#42446, UDF(rawPrediction#42446) AS probability#42454, UDF(rawPrediction#42446) AS prediction#42463]
+- *(1) Project [machineID#42203L, dt_truncated#42204, label_e#42242, features#42329, indexedLabel#42432, indexedFeatures#42439, UDF(indexedFeatures#42439) AS rawPrediction#42446]
 +- *(1) Project [machineID#42203L, dt_truncated#42204, label_e#42242, features#42329, indexedLabel#42432, UDF(features#42329) AS indexedFeatures#42439]
 +- *(1) Project [machineID#42203L, dt_truncated#42204, label_e#42242, features#42329, UDF(cast(label_e#42242 as string)) AS indexedLabel#42432]
 +- *(1) Project [machineID#42203L, dt_truncated#42204, volt_rollingmean_12#42205, rotate_rollingmean_12#42206, pressure_rollingmean_12#42207, vibration_rollingmean_12#42208, volt_rollingmean_24#42209, rotate_rollingmean_24#42210, pressure_rollingmean_24#42211, vibration_rollingmean_24#42212, volt_rollingmean_36#42213, vibration_rollingmean_36#42214, rotate_rollingmean_36#42215, pressure_rollingmean_36#42216, volt_rollingstd_12#42217, rotate_rollingstd_12#42218, pressure_rollingstd_12#42219, vibration_rollingstd_12#42220, volt_rollingstd_24#42221, rotate_rollingstd_24#42222, pressure_rollingstd_24#42223, vibration_rollingstd_24#42224, volt_rollingstd_36#42225, rotate_rollingstd_36#42226, ... 17 more fields]
 +- *(1) FileScan parquet default.scoring_input[machineID#42203L,dt_truncated#42204,volt_rollingmean_12#42205,rotate_rollingmean_12#42206,pressure_rollingmean_12#42207,vibration_rollingmean_12#42208,volt_rollingmean_24#42209,rotate_rollingmean_24#42210,pressure_rollingmean_24#42211,vibration_rollingmean_24#42212,volt_rollingmean_36#42213,vibration_rollingmean_36#42214,rotate_rollingmean_36#42215,pressure_rollingmean_36#42216,volt_rollingstd_12#42217,rotate_rollingstd_12#42218,pressure_rollingstd_12#42219,vibration_rollingstd_12#42220,volt_rollingstd_24#42221,rotate_rollingstd_24#42222,pressure_rollingstd_24#42223,vibration_rollingstd_24#42224,volt_rollingstd_36#42225,rotate_rollingstd_36#42226,... 16 more fields] Batched: true, Format: Parquet, Location: InMemoryFileIndex[dbfs:/user/hive/warehouse/scoring_input], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<machineID:bigint,dt_truncated:timestamp,volt_rollingmean_12:double,rotate_rollingmean_12:d...

In [22]:
predictions.write.mode('overwrite').saveAsTable(dbutils.widgets.get("results_data"))

# Conclusion